# Import delle librerie utilizzate nel progetto

In [ ]:
import os
from math import *
import numpy as np
import pandas as pd
from QuantLib import *
from datetime import date, datetime
import warnings
warnings.filterwarnings('ignore')

path = 'data.xlsx'

# Import delle classi e le funzioni costruite ad hoc per il progetto

In [ ]:
# from classes import 
from functions import ammortamento_francese, ammortamento_italiano, ammortamento_bullet, market_data, estimation_rate_function, discount_rate_function

# Inserimento degli Input

In [ ]:
try:
  underlying = input('Inserire il sottostante: ')
  if underlying == '':
    underlying = 'EUR3M'
except:
  underlying = 'EUR3M'

try:
  amortization_type = input('Inserire il piano di ammortamento: ')
  if amortization_type == '':
    amortization_type = 'Bullet'
except:
  amortization_type = 'Bullet'

try:
  notional = float(input('Inserire il nozionale: '))
except:
  notional = 100000.0

try:
  amortization_rate = float(input('Inserire il tasso annuo: '))
except:
  amortization_rate = .05

try:
  maturity_label = input('Inserire la durata del finanziamento in label ("M" per i mesi e "Y" per gli anni): ')
  if maturity_label == '':
     maturity_label = '5Y'
except:
  maturity_label = '5Y'

try:
  ref_date = Date(input('Inserire la data iniziale del finanziamento in formato dd/mm/yyyy: '),'%d/%m/%Y')
  if ref_date == '':
    ref_date = Date(date.today().day,date.today().month,date.today().year)
except:
  ref_date = Date(date.today().day,date.today().month,date.today().year)

try:
  tenor = int(input('Inserire il periodo che intercorre tra le rate in numero di mesi: '))
except:
  tenor = 3

try:
  business_convention = input('Inserire la convenzione:')
  if business_convention == '' or business_convention == 'ModifiedFollowing':
    business_convention = ModifiedFollowing
except:
  business_convention = ModifiedFollowing

try:
  date_gen_rule = input('Inserire la regola per il calcolo delle date:')
  if date_gen_rule == '' or date_gen_rule == 'Forward':
    date_gen_rule = DateGeneration.Forward
  else: 
    date_gen_rule = DateGeneration.Backward
except:
  date_gen_rule = DateGeneration.Forward

try:
  rule_fixing = input('Inserire la regola per le fixing date: ')
  if rule_fixing == '':
     rule_fixing = '-2BD'
except:
  rule_fixing = '-2BD'

# Schedule delle date di pagamento <br> (viene utilizzato solamente il calendario TARGET in quanto ci si aspetta come sottostante solamente l'EURIBOR)

In [ ]:
calendar = TARGET()
start_date = calendar.advance(ref_date,2,Days)
if maturity_label[-1] == 'M':
    maturity_date = calendar.advance(start_date, int(maturity_label.rsplit(maturity_label[-1], 1)[0]), Months)
elif maturity_label[-1] == 'Y':
    maturity_date = calendar.advance(start_date, int(maturity_label.rsplit(maturity_label[-1], 1)[0]), Years)
Tenor = Period(tenor, Months)
endofmonth = False
terminationDateConvention = business_convention
df = pd.DataFrame({'startdate':list(Schedule(start_date,maturity_date,Tenor,calendar,business_convention,terminationDateConvention,date_gen_rule,endofmonth))[:-1],'enddate':list(Schedule(start_date,maturity_date,Tenor,calendar,business_convention,terminationDateConvention,date_gen_rule,endofmonth))[1:]})

In [ ]:
# Calcolo numero delle rate totali dell'ammortamento
if maturity_label[-1] == 'M':
  rate_totali = int(12/tenor)
elif maturity_label[-1] == 'Y':
  rate_totali = int((12/tenor) * int(maturity_label.rsplit(maturity_label[-1], 1)[0]))

# Creazione del piano di ammortamento a seconda di quello selezionato in input

In [ ]:
if (amortization_type == 'Francese'):
  df_output = (ammortamento_francese(df,amortization_rate,tenor,rate_totali,notional))
elif (amortization_type == 'Italiano'):
  df_output = (ammortamento_italiano(df,amortization_rate,tenor,rate_totali,notional))
elif (amortization_type == 'Bullet'):
  df_output = (ammortamento_bullet(df,amortization_rate,tenor,rate_totali,notional))
print (df_output)

# Calcolo delle Fixing Date utilizzate poi per la stima dei tassi forward

In [ ]:
column_list = ['fixing date','fixing start','fixing end']
for col in column_list:
  df_output[col] = np.zeros

for i, date in enumerate(df_output['startdate']):
  if rule_fixing == '-2BD':
    date_fix = calendar.advance(date, -2, Days)
  else:
    date_fix = calendar.advance(date, -1, Months)
    date_fix = calendar.endOfMonth(date_fix)
    date_fix = calendar.advance(date_fix, -1, Days)

  df_output['fixing date'].iloc[i] = date_fix
  df_output['fixing start'].iloc[i] = calendar.advance(date_fix, 2, Days)
for t, date in enumerate(df_output['fixing start']):
  df_output['fixing end'].iloc[t] = calendar.advance(date, tenor, Months, ModifiedFollowing)

In [ ]:
df_output

# Converto le date della curva importata in Date QuantLib

In [ ]:
market_dict = market_data(path)
for i, d in enumerate(market_dict['ESTIMATION'].zero_curve_rate_df['DATE']):
  market_dict['ESTIMATION'].zero_curve_rate_df['DATE'].iloc[i] = Date(d.day,d.month,d.year)
for i, d in enumerate(market_dict['DISCOUNT'].zero_curve_rate_df['DATE']):
  market_dict['DISCOUNT'].zero_curve_rate_df['DATE'].iloc[i] = Date(d.day,d.month,d.year)

# Chiamata della funzione per l'interpolazione dei forward rate

In [ ]:
estimation_rate_function(market_dict['ESTIMATION'].zero_curve_rate_df,df_output['fixing start'], df_output['fixing end'], ref_date)

# Chiamata della funzione per l'interpolazione dei discount rate

In [ ]:
discount_rate_function(market_dict['DISCOUNT'].zero_curve_rate_df,ref_date,df_output)